# Notebook 3 — Minimal API logic + end-to-end calls (no Docker)

**Goal:** small FastAPI app (in-notebook), recommend unseen, grade with baseline, save attempt; call it with TestClient.

In [1]:
import os 
os.chdir(os.getcwd().split('\\notebooks')[0])
main = os.getcwd()

In [ ]:
# 1) Setup FastAPI + helpers (reuse the same DB from Notebook 1)
import sqlite3, time, json
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from fastapi.testclient import TestClient
from pathlib import Path

DB_PATH = main + "\\data\\temporal\\exams.db"
def con():
    c = sqlite3.connect(DB_PATH); c.row_factory = sqlite3.Row; return c

app = FastAPI(title="Math Trainer (Notebook)", version="0.1")

In [ ]:
# 2) Tiny data helpers
def get_user_id(username: str) -> int:
    with con() as c:
        cur = c.cursor()
        cur.execute("INSERT OR IGNORE INTO users(username) VALUES(?)", (username,))
        cur.execute("SELECT user_id FROM users WHERE username=?", (username,))
        return int(cur.fetchone()["user_id"])

def fetch_question(ex_id: str):
    with con() as c:
        cur = c.cursor()
        cur.execute("""
          SELECT q.exercise_id, q.question, q.solution, q.topic_pred AS topic,
                 e.exam_type, e.date
          FROM questions q LEFT JOIN exams e ON e.exam_id=q.exam_id
          WHERE q.exercise_id=?""", (ex_id,))
        r = cur.fetchone()
        return dict(r) if r else None

def list_unseen(uid: int, k=5):
    with con() as c:
        cur = c.cursor()
        cur.execute("""
          SELECT q.exercise_id, q.topic_pred AS topic, e.date, e.exam_type
          FROM questions q LEFT JOIN exams e ON e.exam_id=q.exam_id
          WHERE q.exercise_id NOT IN (SELECT exercise_id FROM attempts WHERE user_id=?)
          ORDER BY e.date ASC LIMIT ?""", (uid, k))
        return [dict(x) for x in cur.fetchall()]

def save_attempt(uid: int, ex_id: str, result: dict, student_answer: str):
    with con() as c:
        c.execute("""
          INSERT INTO attempts(ts,user_id,exercise_id,score,correct,reasons,hint)
          VALUES(?,?,?,?,?,?,?)
        """, (time.time(), uid, ex_id,
              float(result["score"]), int(result["score"]>=GRADE_THRESHOLD),
              result["reasons"], result["hint"]))

In [ ]:
# 3) Baseline grader (reuse from Notebook 2 quickly)
import re, numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

_word = re.compile(r"[a-zA-Z]+")
def norm(s): return " ".join(_word.findall((s or "").lower()))
def cosine_tfidf(a,b):
    vec = TfidfVectorizer(min_df=1, max_df=0.95, ngram_range=(1,2))
    X = vec.fit_transform([norm(a), norm(b)]).toarray()
    return float((X[0]*X[1]).sum()/(np.linalg.norm(X[0])*np.linalg.norm(X[1])+1e-9))
def jaccard(a,b):
    A,B=set(norm(a).split()), set(norm(b).split()); 
    return float(len(A&B)/max(1,len(A|B)))
def baseline_grade(solution, student):
    c = cosine_tfidf(solution, student); j = jaccard(solution, student)
    score = 0.7*c + 0.3*j
    return {"score":float(score), "reasons":f"cos={c:.2f}, jac={j:.2f}", "hint":"Refuerza definiciones clave."}

GRADE_THRESHOLD = 0.60  # ← replace with your tuned value from Notebook 2

In [ ]:
# 4) Schemas & endpoints
class AttemptIn(BaseModel):
    username: str
    exercise_id: str
    answer: str

@app.get("/health")
def health():
    return {"ok": True, "db": str(DB_PATH)}

@app.get("/questions/next")
def next_q(username: str, k: int = 3):
    uid = get_user_id(username)
    return list_unseen(uid, k=k)

@app.get("/questions/{exercise_id}")
def get_q(exercise_id: str):
    q = fetch_question(exercise_id)
    if not q: raise HTTPException(404, "Unknown exercise_id")
    return q

@app.post("/attempts")
def attempts(body: AttemptIn):
    uid = get_user_id(body.username)
    q = fetch_question(body.exercise_id)
    if not q: raise HTTPException(404, "Unknown exercise_id")
    result = baseline_grade(q["solution"], body.answer)
    save_attempt(uid, body.exercise_id, result, body.answer)
    return {
        "exercise_id": body.exercise_id,
        "score": result["score"],
        "correct": bool(result["score"]>=GRADE_THRESHOLD),
        "reasons": result["reasons"],
        "hint": result["hint"]
    }

In [ ]:
# 5) End-to-end calls (no server needed): FastAPI TestClient
client = TestClient(app)

print(client.get("/health").json())

nxt = client.get("/questions/next", params={"username":"alice","k":2}).json()
nxt

In [ ]:
# 6) Fetch one question and submit an answer
ex_id = nxt[0]["exercise_id"]
print(client.get(f"/questions/{ex_id}").json()["question"])

good = "Use L<1 and the contraction principle to guarantee a unique fixed point."
bad  = "No idea."

print("GOOD:", client.post("/attempts", json={"username":"alice","exercise_id":ex_id,"answer":good}).json())
print("BAD :", client.post("/attempts", json={"username":"alice","exercise_id":ex_id,"answer":bad}).json())

In [ ]:
# 7) Quick peek at attempts saved
import sqlite3
con = sqlite3.connect(DB_PATH); con.row_factory = sqlite3.Row
rows = con.execute("SELECT exercise_id, score, correct, ts FROM attempts ORDER BY ts DESC LIMIT 5").fetchall()
con.close()
[dict(r) for r in rows]